# Copies

copies means more memory hogging and that will remain true. 
Hence, Mutate!

# Pydantic `model_copy`
```python
def model_copy(self: Model, *, update: dict[str, Any] | None = None, deep: bool = False) -> Model
``` 

- The update dictionary here is of help as it takes a `key = attr_name` and `value = value_to_be_replaced`
- Notice it also has a `deep = False` param. We'll come to it later.

If mutation helps in using less memory, why create a model_copy. Well it seems like `Pydantic` is **aware** of that
and mirrors python reference counting under the hood.

So it's something like this : 


In [6]:

from src.basemodels import ImmutableModel

class A(ImmutableModel):
    a: int
    b: float
    c: tuple[int, ...] # the ... means multiple

a = A(a = 1, b = 2.2, c = (1,2,3))
   


Here `a` is Like this

    1       2.2     (1,2,3)
    a.a     a.b       a.c

Now when I do,


In [7]:
c = a.model_copy(update={'b': 4.4})
print(c)

a=1 b=4.4 c=(1, 2, 3)



Now `a` is becomes this

    1       2.2     (1,2,3)
    a.a     a.b       a.c
    
     |      4.4        |
    c.a     c.b       c.c                   

As you can see here 
- `a.b` still references `2.2` but the 
- `b.c` references `4.4`
- `c.a and c.c` references `a.a and a.c` respectively

This means there is no separate memory block being created for `c.a and c.c`, just a reference. Meanwhile, for `c.b` a new block is assigned with value `4.4`  and chained together with `c.a and c.c` to form `c`.


### deep: boolean = False
This is where the `deep` attribute comes into picture.
If you set the `deep` = `True` during your model_copy, the `c` becomes : 

    1       2.2     (1,2,3)
    a.a     a.b       a.c
    
     1      4.4      (1,2,3)
    c.a     c.b       c.c                   

So an entirely new blocks of `c.a and c.c` are created. This means even if `a` changes, `c` will remain intact. So it means a extra memory allocation to the same values. What I suggest is:
- Use `deep = False` (default) when you know for sure that `a` is not going to change or even if it changes, you are only concerned with the attributes in `update` list. 
- Use `deep = True` when you are also concerned about the other attributes of `a` and want them to be in the same state as they are at the time of generating copy

## new : A step further

```python
def new(self, key: str, value: T, deep: bool = False) -> Self:
```

In the `Immutable` model class, we also have a `new` method that returns an instance of itself (Subclass-able). The reason this method was added was because, let's assume we have a class


In [8]:
class A(ImmutableModel):
    a: int
    b: float
    c: list[int]

class B(ImmutableModel):
    a: A
    b: float
    c: str 
    
b = B(a = c.model_dump(), b=22.24, c = "new start")
print(b)

a=A(a=1, b=4.4, c=[1, 2, 3]) b=22.24 c='new start'


Now if I want to create a model_copy but only change `b.a.b = 8.8` instead. Rest references should remain same. I cannot do that with `model_copy`. With `model_copy` I have to recreate `a` with updated `a.b` and assign that to the `model_copy` of b. Like follows


In [9]:
a = A(a=1, b = 8.8, c = [1,2,3])
d = b.model_copy(update={'a': a})
print(d)

a=A(a=1, b=8.8, c=[1, 2, 3]) b=22.24 c='new start'


This is much painful as it's creating a new object `a` with same other attributes which can instead be kept as references.
Hence, what `new` does is that it takes 
- `key` = `a.b`. This is `model.attribute` which can be chained too like `model.model.model...model.attribute`
- `value` = Value that is to be set to the attribute. Please make sure to **respect** the type

Moreover, the `deep` propagation is preserved. Although in this case it applies to all the `model`s.
The heart of the `new` method is `_copy` method that has the same blueprint.

The reason it is kept separate is to have different `mixin` implementations for copying beyond `pydantic` models and to even enhance the within model copying like deep copying only from a specific level. 

In [10]:
print(f"d initially: {d}")
e = d.new(key="a.b", value=2.2)
print(e)
d.a.c.append(100)
print(f'd changed: {d}')
print(f"e affected: {e}")

print("DEEP:  TRUE =================")
# Deep propagation is preserved

f = d.new(key="a.b", value="3.3", deep=True)
print(f"f created: {f}")
d.a.c.append(100)
print(f"d changed: {d}")
print(f"f unchanged: {f}")



d initially: a=A(a=1, b=8.8, c=[1, 2, 3]) b=22.24 c='new start'
a=A(a=1, b=2.2, c=[1, 2, 3]) b=22.24 c='new start'
d changed: a=A(a=1, b=8.8, c=[1, 2, 3, 100]) b=22.24 c='new start'
e affected: a=A(a=1, b=2.2, c=[1, 2, 3, 100]) b=22.24 c='new start'
DEEP:  TRUE =================
f created: a=A(a=1, b='3.3', c=[1, 2, 3, 100]) b=22.24 c='new start'
d changed: a=A(a=1, b=8.8, c=[1, 2, 3, 100, 100]) b=22.24 c='new start'
f unchanged: a=A(a=1, b='3.3', c=[1, 2, 3, 100]) b=22.24 c='new start'
